In [ ]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import json

In [3]:
explainers = BeautifulSoup(requests.get("http://theconversation.com/au/factcheck").text)

C:\Anaconda3\lib\site-packages\bs4\__init__.py:166: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

To get rid of this warning, change this:

 BeautifulSoup([your markup])

to this:

 BeautifulSoup([your markup], "lxml")

  markup_type=markup_type))


In [4]:
headlines = explainers.find_all('h2')

In [5]:
results = list()

In [6]:
for headline in headlines:
    thisline = dict()
    try:
        link = "http://theconversation.com" + headline.a['href']
        thisline['headline'] = headline.text
        thisline['content'] = requests.get(link).text
        thisline['link'] = link
    except:
        pass
    results.append(thisline)

In [7]:
abcexplainers = BeautifulSoup(requests.get('http://www.abc.net.au/news/factcheck/zombies/').text)

C:\Anaconda3\lib\site-packages\bs4\__init__.py:166: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

To get rid of this warning, change this:

 BeautifulSoup([your markup])

to this:

 BeautifulSoup([your markup], "lxml")

  markup_type=markup_type))


In [8]:
links = abcexplainers.find_all("a")

In [ ]:
thisline = list()
for link in links:
    if "Fact Check zombie" in link.text:
        abcid = link['href'].split('/')[-1]
        url = "https://content-api-govhack.abc-prod.net.au/v1/" + abcid
        content = requests.get(url).text
        jsondoc = json.loads(content)
        thisline.append(jsondoc)

In [15]:
df = pd.DataFrame(results)

In [16]:
df

,content,headline,link
0,<!DOCTYPE html>\n<!--[if IE 7]> <html class...,Election FactCheck: Does the government spend ...,http://theconversation.com/election-factcheck-...
1,<!DOCTYPE html>\n<!--[if IE 7]> <html class...,Has the Coalition invested an average of $5 bi...,http://theconversation.com/election-factcheck-...
2,<!DOCTYPE html>\n<!--[if IE 7]> <html class...,Has the Coalition cut bulk-billing for patholo...,http://theconversation.com/election-factcheck-...
3,<!DOCTYPE html>\n<!--[if IE 7]> <html class...,"Have 300,000 new jobs been created in the last...",http://theconversation.com/election-factcheck-...
4,<!DOCTYPE html>\n<!--[if IE 7]> <html class...,Is the Australian Sex Party right about religi...,http://theconversation.com/election-factcheck-...
5,<!DOCTYPE html>\n<!--[if IE 7]> <html class...,Does the government spend more on negative gea...,http://theconversation.com/election-factcheck-...
6,<!DOCTYPE html>\n<!--[if IE 7]> <html class...,"Are larger, more frequent storms predicted due...",http://theconversation.com/election-factcheck-...
7,<!DOCTYPE html>\n<!--[if IE 7]> <html class...,"Have 50,000 full-time jobs been lost this year...",http://theconversation.com/election-factcheck-...
8,<!DOCTYPE html>\n<!--[if IE 7]> <html class...,Is it true no solely managed Commonwealth fish...,http://theconversation.com/election-factcheck-...
9,<!DOCTYPE html>\n<!--[if IE 7]> <html class...,Has $100 billion been added to Australia’s nat...,http://theconversation.com/election-factcheck-...


In [18]:
df.to_pickle('explainers.pickle')